# Image Classification visualizer

https://github.com/henripal/maps/blob/master/nbs/big_resnet50-interpret-gradcam-dogs.ipynb
https://github.com/henripal/maps/blob/master/nbs/big_resnet50-interpret-gradcam-sats.ipynb

In this notebook we will take the model from CatDogChickenBaby and play with some visualisations.
The visualisations come from the [grad-cam](https://arxiv.org/abs/1610.02391) paper and would not have been possible (or even known) without the great work of [henripal](http://henripal.github.io/).

Lets begin!

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
import fastai
from fastai import *
from fastai.vision import *
import re
import scipy.ndimage

In [8]:
print(torch.cuda.is_available())
print(fastai.__version__)

True
1.0.22


In [ ]:
learn.load('stage-2')

## Grad cam setup

See the paper for details. The idea is to take the outputs of the last "convolutional layer", here a stack of 7x7 feature maps, then weigh them by their importance to the given class using an average of the gradients of those layers wrt the class score vector.

We therefore need to set up two hooks:

for the feature maps
for the gradient
Then we linearly combine the feature maps using a reduced version of the gradient to obtain the heatmap.

In [ ]:
# we are going to use the output of the last Bottleneck as feature maps
# this is hardcoded for resnet50
target_layer = learn.model[0][7][2]
target_layer

In [ ]:
# hook for the feature maps
fmap_hook = fastai.callbacks.hook_output(target_layer)

In [ ]:
# hook for the gradients 
def gradient_torch_hook(self, grad_input, grad_output):
    return grad_input

gradient_hook = fastai.callbacks.Hook(target_layer, gradient_torch_hook, is_forward=False)

# Grad cam

Our hooks are setup, let's run the forward and backward passes to get what we need.

In [ ]:
# we start by extracting a batch then choosing one image and label from that batch
# i is the index of the chosen image
i = 13
image_batch, label_batch = next(iter(learn.data.train_dl))
image = image_batch[i].reshape(1, 3, 224, 224)
label = label_batch[i].reshape(-1)

In [ ]:
learn.model.eval();

In [ ]:
# running the forward pass
learn.model.zero_grad();
out = learn.model(image)

# we need to set the gradients at the output to ones at the predicted class
# and zero everywere else
onehot = torch.zeros(learn.data.c)
onehot[torch.argmax(out)] = 1.0

# we then backprop from there
out.backward(gradient=onehot.reshape(1, -1).cuda(), retain_graph=True)

In [ ]:
gradients = next(iter(gradient_hook.stored))
gradients.shape

We've now run our forward and backward passes, we only need to combine the gradients and feature maps to obtain our heat map

In [ ]:
gradient_linearization = gradients.cpu().numpy().sum((2, 3)).reshape(-1)
fmaps = fmap_hook.stored.cpu().numpy()
print(fmaps.shape)
fmaps = fmaps.reshape(2048, 7, 7)

In [ ]:
# relu on the heatmap
heatmap = np.maximum(0, np.einsum('i, ijk',gradient_linearization, fmaps))

# we now upsample the heatmap so we can overlay it on our original image
upsampled = scipy.ndimage.zoom(heatmap, 32)
upsampled = (upsampled - np.min(upsampled))/(np.max(upsampled) - np.min(upsampled))

In [ ]:
def image_from_tensor(imagetensor):
    numpied = torch.squeeze(imagetensor)
    numpied = np.moveaxis(numpied.cpu().numpy(), 0 , -1)
    numpied = numpied - np.min(numpied)
    numpied = numpied/np.max(numpied)
    return numpied

In [ ]:
plt.imshow(image_from_tensor(image))
plt.imshow(upsampled, alpha=.8)
plt.gca().set_axis_off()

## Guided backprop

So, our heatmap above is nice but unfortunately not very fine-grained. We are constrained by the resolution of the feature maps (in our example 7x7).

A first idea would be to naively compute the backprop the gradients of our "forced score" all the way to the image, and see which pixel has higher gradients. So let's do that:

In [ ]:
# we ask the model to compute the gradients wrt the image:
image.requires_grad_()

# and then perform our forward and backward passes, forcing the gradient to be 1 for our predicted class
out = learn.model(image)
out.backward(gradient=onehot.reshape(1, -1).cuda())

In [ ]:
plt.imshow(image_from_tensor(image.grad))
plt.gca().set_axis_off()

That looks horrifying. This because the various neurons [interfere](https://en.wikipedia.org/wiki/Wave_interference) with each other during the backpropagation.

The [Strving for Simplicity: the All Convolutional Net paper](https://arxiv.org/abs/1412.6806) introduced the concept of Guided Backpropagation. The idea, to avoid interference is to simply retain the positive contributions to the gradient while back-propaagating, thereby avoiding all interference.

The idea is to clip all negative gradients at each ReLU layer. We will do this using the Hooks module in the fastai library

In [ ]:
## this callback will make all gradients positive during backprop
def clamp_gradients_hook(module, grad_in, grad_out):
    for grad in grad_in:
        torch.clamp_(grad, min=0.0)

In [ ]:
## we inventory all Relu Modules
relu_modules = [module[1] for module in learn.model.named_modules() if str(module[1]) == "ReLU(inplace)"]
len(relu_modules)

In [ ]:
## and register our hooks with fastai
hooks = fastai.callbacks.Hooks(relu_modules, clamp_gradients_hook, is_forward=False)

Our hooks are registered, we will now backprop and observe the result:

In [ ]:
learn.model.eval()
image.requires_grad_()
learn.model.zero_grad()
out = learn.model(image)

In [ ]:
out.backward(gradient=onehot.reshape(1, -1).cuda(), retain_graph=True)

In [ ]:
plt.imshow(image_from_tensor(image.grad))
plt.gca().set_axis_off()

To make sure we are only looking at relevant areas of the image, we will multiply this by the heatmap computed using Grad CAM:

In [ ]:
gbp =  image_from_tensor(image.grad)

In [ ]:
prod = np.einsum('ijk, ij->ijk',gbp, upsampled)

In [ ]:
plt.imshow(prod)
plt.gca().set_axis_off()

Done?